In [21]:
import pandas as pd
from sklearn.model_selection import train_test_split
from skopt.space import Real, Integer, Categorical
from skopt import BayesSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
# Cell 1: Import Libraries
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, LSTM, Dense, Dropout, Input, Reshape
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import optuna


In [75]:
dataset = pd.read_excel('data_pattern_4_inclusive_balanced.xlsx')

In [76]:
dataset['class'].value_counts()
# dataset.columns

class
afrobeat         51
neworleans       51
disco            51
gospel           51
highlife         51
afrocuban        51
punk             51
middleeastern    51
reggae           48
breakbeat        48
country          48
funk             48
latin            48
soul             44
blues            43
pop              42
jazz             42
rock             42
hiphop           38
Name: count, dtype: int64

In [78]:
X = dataset.filter(like='feature_').values
y = dataset['class'].values

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
)


In [79]:
import torch
import torch.nn as nn

class GenreRNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes, dropout):
        super(GenreRNN, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        h_0 = torch.zeros(self.lstm.num_layers, x.size(0), self.lstm.hidden_size).to(x.device)
        c_0 = torch.zeros(self.lstm.num_layers, x.size(0), self.lstm.hidden_size).to(x.device)
        
        out, _ = self.lstm(x, (h_0, c_0))
        out = self.fc(out[:, -1, :])  # Use the last time-step output
        return out


In [ ]:
import optuna
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import accuracy_score

def objective(trial):
    # Hyperparameters
    hidden_size = trial.suggest_int('hidden_size', 16, 128, step=16)
    num_layers = trial.suggest_int('num_layers', 1, 3)
    dropout = trial.suggest_float('dropout', 0.1, 0.5)
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-2)
    
    # Model
    model = GenreRNN(input_size=64, hidden_size=hidden_size, num_layers=num_layers, 
                      num_classes=len(set(y_train)), dropout=dropout).to(device)
    
    # Loss and Optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    # Training Loop
    num_epochs = 20
    batch_size = 32
    train_loader = DataLoader(TensorDataset(torch.tensor(X_train, dtype=torch.float32), 
                                            torch.tensor(y_train, dtype=torch.long)), 
                              batch_size=batch_size, shuffle=True)

    model.train()
    for epoch in range(num_epochs):
        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

    # Evaluation
    model.eval()
    with torch.no_grad():
        X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
        y_test_tensor = torch.tensor(y_test, dtype=torch.long).to(device)
        y_pred = model(X_test_tensor).argmax(dim=1).cpu().numpy()
        accuracy = accuracy_score(y_test, y_pred)
    
    return accuracy

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)
